# 1. 최단거리 알고리즘

## 1) 다익스트라

- 특정한 노드에서 출발하여 다른 노드로 가는 각각의 최단 경로를 구해주는 알고리즘
- 그리디 알고리즘 : 매번 가장 비용이 적은 노드를 선택하여 임의의 과정을 반복
- 1차원 리스트 : 각 노드에 대한 현재까지의 최단 거리 정보를 항상 1차원 리스트에 저장하고 계속 갱신함
- int(1e9) : 1차원 리스트를 초기화하는 값은 무한대로 지정, 이 때, 파이썬은 1e9를 실수형으로 처리하기에 int형 변환


### 구현 방식
1. 출발 노드를 설정
2. 최단 거리 테이블을 초기화
3. 방문하지 않은 노드 중에서 최단 거리가 짧은 노드를 선택
4. 해당 노드를 거쳐 다른 노드로 가는 비용을 계산하여 최단 거리 테이블을 갱신
5. 위 과정에서 3, 4를 반복

### 1-1) 간단한 다익스트라 알고리즘

- 시간 복잡도 : O(V^2) [V = 노드의 개수]
- 매 단계마다 방문하지 않은 노드 중에서 최단 거리가 가장 짧은 노드를 선택하기 위해 매 단계마다 1차원 리스트의 모든 원소를 순차탐색함
- 다익스트라의 경우 입력되는 데이터의 수가 많을 수 있다.
- 따라서 input() 함수를 sys.stdin.readline()으로 치환하여 사용한다.
- DFS, BFS와 같이 노드의 번호를 맞춰주기 위해 리스트의 크기는 V+1로 선언한다.

In [ ]:
import sys

input = sys.stdin.readline
INF = int(1e9)

# 노드의 개수, 간선의 개수 입력
n, m = map(int, input().split())
# 시작 노드 번호
start = int(input())
# 각 노드에 연결되어 있는 노드에 대한 정보를 담는 리스트
graph = [[] for i in range(n+1)]
# 방문여부 기록 리스트
visited = [False] * (n+1)
# 최단 거리 테이블을 무한으로 초기화
distance = [INF] * (n+1)

# 간선정보 입력받기
for _ in range(m):
    a, b, c = map(int, input().split())
    graph[a].append((b, c))
    

# 방문하지 않은 노드 중에서 가장 길이가 짧은 노드 반환
def get_smallest_node():
    min_value = INF
    index = 0    
    for i in range(1, n+1):
        if distance[i] < min_value and not visited[i]:
            min_value = distance[i]
            index = i
    return index

def dijkstra(start):
    # 시작 노드에 대해 초기화
    distance[start] = 0
    visited[start] = True
    
    # 인접 노드의 거리 초기화
    for j in graph[start]:
        distance[j[0]] = j[1]
    
    # 시작 노드를 제외한 n - 1 개의 노드에 대해 반복
    for i in range(n-1):
        # 현재 최단 거리가 가장 짧은 노드를 꺼내서 방문처리
        now = get_smallest_node()
        visited[now] = True
        
        # 현재 노드와 연결된 다른 노드를 확인
        for j in graph[now]:
            cost = distance[now] + j[1]
            
            # 현재 노드를 거쳐서 다른 노드로 이동하는 거리가 더 짧은 경우
            if cost < distance[j[0]]:
                distance[j[0]] = cost
                
dijkstra(start)

# 모든 노드로 가기 위한 최단 거리를 출력
for i in range(1, n + 1):
    # 도달할 수 없는 경우 INFINITY 출력
    if distance[i] == INF:
        print("INFINITY")
    else:
        print(distance[i])

- 본 알고리즘으로는 노드의 개수가 10,000개 넘어가면 사용할 수 없음 (시간초과)

### 1-2) 개선된 다익스트라 알고리즘

- 시간복잡도 : O(ElogV) [E : 간선의 개수, V : 노드의 개수]
- 기존 알고리즘 대비 비용이 가장 작은 간선을 찾아내는 알고리즘을 개선한 방법 - HEAP 구조 사용
- 현재 가장 가까운 노드를 저장하기 위한 목적으로 우선순위 큐가 추가됨

#### Heap (힙)
- 우선순위 큐를 구현하기 위해 사용되는 자료구조 중 하나
- 우선순위 큐 : 우선순위가 가장 높은 데이터를 가장 먼저 삭제 - 파이썬에서는 heapq 모듈로 사용할 수 있음
- 내부적으로 Max Heap 또는 Min Heap을 사용함 - 파이썬에서는 최소힙으로 구현되어 있음 (C++ : Max Heap)
- 최소힙을 최대힙처럼 사용하기 위해선, 간선의 비용 정보를 음수로 변환하여 저장하면 됨
- 파이썬 라이브러리는 우선순위 큐에 들어가는 경우, 이를 tuple로 받아서, 첫번째 인자를 비용으로 인식하고 기준으로 정렬함

In [ ]:
import heapq
import sys
input = sys.stdin.readline
INF = int(1e9)

n, m = map(int, input().split())
start = int(input())
graph = [[] for i in range(n+1)]
distance = [INF] * (n+1)

# 간선 정보 입력
for _ in range(m):
    a, b, c = map(int, input().split())
    graph[a].append((b, c)) # a에서 b로 가는데, c만큼의 비용이 들어감
    
def dijkstra(start):
    q = []
    # 시작노드의 초기값 거리를 0으로 초기화하여 큐에 삽입
    heapq.heappush(q, (0, start))
    distance[start] = 0
    
    # 우선순위 큐가 비어버릴 떄까지 반복
    while q:
        dist, now = heapq.heappop(q)
        
        # 이미 처리된 노드라면 무시
        if distance[now] < dist:
            continue
    
        # 현재 노드와 인접한 노드를 확인
        for i in graph[now]:
            cost = dist + i[1]
            # 현재 노드를 거쳐서, 다른 노드로 이동하는 거리가 더 짧은 경우
            if cost < distance[i[0]]:
                distance[i[0]] = cost
                heapq.heappush(q, (cost, i[0]))

dijkstra(start)

# 모든 노드로 가기 위한 최단 거리를 출력
for i in range(1, n + 1):
    # 도달할 수 없는 경우 INFINITY 출력
    if distance[i] == INF:
        print("INFINITY")
    else:
        print(distance[i])